In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import richdem as rd

In [ ]:
# TODO: import code

In [ ]:
# Test Direction Calculations

# Replicate results from the paper
# "Cellular Automata for the Flow Simulations on the Earth Surface"
# Cirbus and Podhoranyi (2013)

# Take the sample DEM from the paper
cirbus_dem = np.array(
    [
        [203.83, 201.09, 201.59, 204.31, 200.21, 201.71],
        [204.35, 203.89, 204.13, 200.29, 202.02, 200.20],
        [202.18, 201.27, 203.95, 201.20, 200.46, 201.72],
        [200.74, 202.66, 202.71, 204.34, 200.52, 201.95],
        [201.14, 204.04, 200.64, 203.18, 204.03, 204.35],
        [201.57, 203.51, 200.79, 200.01, 202.00, 202.02]
    ])

# And check our calculated directions with those in the paper
cirbus_directions = np.array([
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0., 103., 251.,   4., 254.,   0.],
       [  0.,  32., 125., 146.,   5.,   0.],
       [  0., 171.,  77., 255.,   2.,   0.],
       [  0., 255., 128., 237., 238.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.]
])

# Are direction calculations correct for all cells?
assert init_direction(cirbus_dem).all() == cirbus_directions.all()

In [ ]:
# Test flow accumulation matrix calculation using two predictable cases

# Case 1: A downhill slope. 
# Expectation: Lowest cell has the largest accumulation.

down_stream = np.zeros((5,5))
down_stream[0:4,2] = 128

plt.imshow(
    generate_flow_acc(down_stream, n_iters = 100000, max_visits = 10))


# Case 2: A penrose staircase.
# Expectation: The cells are in a (physically impossible) infinite loop
# so we expect a uniform distribution

# Penrose Staircase (after an Escher binge in quarantine)
penrose = np.zeros((5,5))

penrose[1,1:4] = 2**5
penrose[1:4,3] = 2**7
penrose[3,1:4] = 2**3
penrose[2:4,1] = 2

penrose_acc = generate_flow_acc(penrose, n_iters = 20000, max_visits = 10)
plot = orbit_dem(penrose_acc)

In [ ]:
# Test gradient calculation against richdem module
path = './media/beauford.npz'
with np.load(path) as data:
    dem = data['beauford']

dem = rd.rdarray(dem[400:800,400:800], no_data=-9999)
slope = rd.TerrainAttribute(dem, attrib='slope_degrees')

slopes_test = init_slope(dem)

plt.hist(np.array(slope.flatten(), dtype = np.int8), alpha =  0.5, bins = 40)
plt.hist(np.array(slopes_test.flatten(), dtype = np.int8), alpha = 0.5, bins = 40)
plt.show()

# Assert no angles are greater than 90
assert np.sum(np.where(slopes_test > 90, 1, 0)) == 0

# check accuracy of slope measurement equal between two arrays
np.sum(slopes_test == slope)/slopes_test.size

In [ ]:
# Test the exactness of the gradient calculation on toy basin data
basin = create_basin()
basin_arr = np.array(basin[...,2], dtype = np.float16)

rd_basin = rd.rdarray(basin[...,0], no_data=-9999)
rd_deg = np.array(rd.TerrainAttribute(rd_basin, attrib='slope_degrees'), dtype = np.float16)

rd.all() == basin_arr.all()

# Old Update Rules

In [ ]:

# liquid loss = inflitration
depth = np.linspace(0,10,100)
s = np.linspace(0,100,100)
n = 0.02
w = 10

# Flow rate 
def fluid_velocity(depth = 4, s=np.pi, n = 0.2):
    """Params
        cell  (array[4]): a sim cell
        depth[m]   (int): depth of water column at cell
        s[deg]   (float): slope of cell
        n        (float): Manning roughness coefficient."""

    #depth = cell[0] 
    #s     = cell[2] 

    return depth**(2/5)*np.sqrt(s) / n


def time_through_cell(cell, width = 10):
    """Params
        v    (float): calculated fluid_velocity through cell
        width  (int): width of cell/raster resolution"""

    v = fluid_velocity(cell)

    return width/v

# What is this referring to
def volume_entering_cell(depth, width, v, t = 1):
    return depth * width * v * t

# Meet flooding conditions
def is_flooded(cell, t = 1):
    """check if cell is flooded
    Params
    Returns
        flooded? (bool)"""
    return time_through_cell(cell) > t


def infiltration_velocity(Ks = 2, Hf = 1, F = 2, Md = 3):
    """
    Params
        Ks(mmh^-1) : hydraulic conductivity
        Hf(mm)     : suction lift
        Md(mm)     : soil moisture deficit
        F          : total depth of infiltration
    """
    # How fast water seeps into the soil
    # infiltration_velocity << fluid_velocity
    return Ks*(1 + Hf*Md/F)